In [1]:
include("../code/julia/ha-trade-environment.jl")
include("../code/julia/ha-trade-solution.jl")
include("../code/julia/ha-trade-helper-functions.jl")
include("../code/julia/ha-trade-elasticity.jl")
include("../code/julia/ha-trade-efficient.jl")
include("../code/julia/static-trade-environment.jl")
include("../code/julia/gravity-tools.jl")

using MINPACK
using Plots
using CSV
using DataFrames

### The Curios Case of the Log-model

Here I walk through some of the properties that of the log model in a two country setting.



In [14]:
γ = 1.0 # Setting \gamma = 1.0 is when u(c) = log(c)

1.0

In [16]:
# The rest of the parameters

σϵ = 0.25
Ncntry = 2 # number of countries

# this setups up parameters on the household side
hh_prm = household_params(Ncntry = Ncntry, Na = 100, β = 0.92, γ = γ, ϕ = 0.5, amax = 8.0, σϵ = σϵ);


In [17]:
# here are some simpe country parametrers
TFP = [1.0; 1.0]

L = [1.0; 1.0]

d_ij = 2.05

d = [1.0 d_ij; d_ij 1.0]

# this sets up the country specific paramters
cntry_prm = country_params(Ncntry = Ncntry, L = L, d = d, TFP = TFP);

---

### Find an equilibrium 

Does what it says. Here I'm computing one for financial globalization. 

In [18]:
f(x) = world_equillibrium_FG(exp.(x), hh_prm, cntry_prm)
# this world... function is used to construct zero conditions for
# the finacial globalization case

function f!(fvec, x)

    fvec .= f(x)

end


xguess = [1.0; 1.02]

n = length(xguess)
diag_adjust = n - 1

sol = fsolve(f!, log.(xguess), show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-10,
       )

# This solver works very well in general. Spencer Lyon converted old-school minpack
# routines in C into julia

print(sol)

wage = [exp.(sol.x[1]); 1.0]
R = exp.(sol.x[2])

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     4.866213e-01     0.000000e+00         0.150000
     2     9.842734e-01     1.635380e-03         0.313000
     3     1.528856e-01     1.067457e-03         0.154000
     4     5.314370e-02     3.145013e-05         0.094000
     5     6.869751e-03     9.136320e-06         0.116000
     6     5.572147e-04     1.117923e-07         0.119000
     7     1.953094e-06     6.272382e-10         0.124000
     8     3.885219e-08     7.528764e-15         0.102000
     9     1.702087e-10     2.883283e-18         0.098000
    10     2.735520e-13     5.568670e-23         0.142000
    11     3.275158e-15     1.342411e-28         0.105000
Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [0.0, 0.01980262729617973]
 * Zero: [-3.799705427411952e-9, 0.04047910888855125]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estim

1.0413095553613083

Then here are the results. The first entry is the wage rate which should be 1.0 given the symmetry. The second price is the interest rate, a side note, but the interest rate now is much higher to get people to hold the asset

In [19]:
println(exp.(sol.x))


[0.9999999962002946, 1.0413095553613083]


Then this call computes aggregates given prices

In [20]:
Rsol = [R; R]

Y, tradeflows, A_demand, tradeshare, hh, dist = world_equillibrium(Rsol, wage, hh_prm, cntry_prm, tol_vfi = 1e-10);

# this world_eq...is a core file takes prices and returns a bunch of stuff
# note that hh, dist are objects of dimensiom number of countries, then within
# it has policy functions and distributions state by state

Here we can see the trade share matrix. Importers are rows, columns are exporters. Less trade relative to \gamma > 1

In [21]:
tradeshare

2×2 Matrix{Float64}:
 0.946412   0.0535877
 0.0535877  0.946412

here are the state by state choice probabilities of households in country 1. Consistent with the discussion in the paper, the choice probabilities are exactly the same for all households | even the constrained guys! 

In [22]:
hh[1].πprob[:,:,2]
# guys in country one, a X z, probability of buying from 2

100×10 Matrix{Float64}:
 0.0535877  0.0535877  0.0535877  …  0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877  …  0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877  …  0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 0.0535877  0.0535877  0.0535877     0.0535877  0.0535877  0.0535877
 ⋮                                ⋱                        
 0.0535877  0.05358

---

### Elasticities in the Log model

A result in the paper is with log, elasticities become constant. Let's check this out

In [23]:
cntry = 1 # this is the country I'll look at

p = (wage[1:end] ./ TFP).*d[cntry,:] 
# prices from the perspective of those in that country

hh = solve_household_problem(R, wage[cntry], p, hh_prm, tol = 1e-10);
# solve hh problem at those prices

dist = make_stationary_distribution(hh, hh_prm);
# find stationary distribution


This cell below then constructs the elasticities via numerical differentiation. Then the last two function calls construct the weights and then aggregate.

A note on accuracy. I don't have a closed form for this. But in Mongey and Waugh, we do. And their I found that this approach worked well with the only issues being around states where the household is about to leave the borrowing constraint. This is to be expected per my discussion in the paper regarding how the constraint might show up in the elasticities.

In [24]:
θ = make_θ(cntry, R, wage[cntry], p, hh_prm; points = 3, order = 1)
# makes the micro elasticities for country, cntry, 

ω = make_ω(hh, dist, L[cntry], p, hh_prm)
# makes the expenditure weights

agθ = aggregate_θ(θ, ω, cntry, hh_prm)
# construct the aggregate elasticity. The interesting element here is 
# agθ[cntry]

2-element Vector{Float64}:
  1.0
 -3.999999999982769

**A quick tour of the results:**

The second element above is the aggregate trade elasticity. In this situation it is -4 which is exactly the prediction in the paper, it should equall $1 / \sigma_\epsilon$

Then we can look at the **micro-elasticities**. 

Here is how θ_{ij}^E looks state by state (assets are rows, income shocks are columns) where one sees constant extensive margin elasticities

In [25]:
θ.θπ[:,:,2]

100×10 Matrix{Float64}:
 -3.78565  -3.78565  -3.78565  -3.78565  …  -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565  …  -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565  …  -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
 -3.78565  -3.78565  -3.78565  -3.78565     -3.78565  -3.78565  -3.78565
  ⋮                        

Then one sees the same thing on the **intensive margin**. Everyone has an elasticity of -1, so all cost savings are directed towards more consumption.

In [26]:
θ.θc[:,:,2]

100×10 Matrix{Float64}:
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
  ⋮                             ⋮                      
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -

And then here is the $\theta^I_{ii,j}$, so how much does home consumption on the intensive margin respond. Not at all which is consistent with the theoretical arguments in the paper.

In [28]:
θ.θcii[:,:,2]

100×10 Matrix{Float64}:
  0.0           0.0          0.0          …  -1.36939e-11  0.0
  0.0           0.0          0.0              1.36939e-11  0.0
  0.0           0.0          0.0              1.36939e-11  0.0
  0.0           0.0          0.0             -1.36939e-11  1.36939e-11
  0.0           0.0          0.0              0.0          0.0
  0.0           1.36939e-11  0.0          …   0.0          0.0
  2.73878e-11   2.73878e-11  0.0              0.0          2.73878e-11
  1.36939e-11   0.0          0.0              1.36939e-11  1.36939e-11
  2.73878e-11   0.0          0.0              2.73878e-11  1.36939e-11
  0.0          -2.05408e-11  0.0              2.73878e-11  4.10816e-11
  0.0           0.0          0.0          …   1.36939e-11  4.10816e-11
  0.0           0.0          0.0              0.0          5.47755e-11
 -2.05408e-11   3.42347e-11  0.0              0.0          2.73878e-11
  ⋮                                       ⋱                
  7.53163e-11   0.0          0.0 